In [ ]:
# INSTALLING DEPENDENCIES
pip install --upgrade breeze-connect
pip install backtrader
pip install quantstats

In [ ]:
# API CREDENTIALS  - ICICI DIRECT
user_name ='9847829376'
password='912379'
api_key = "3E1797j@19`305130Q25755pJ574R347"
api_secret = "56I80%80!35DG84w6idp19S9126o9x76"
api_session = '31682059'

In [ ]:
# IMPORTING REUIRED LIBRAR
import pandas as pd
import time
import numpy as np 

from datetime import datetime,timedelta
from breeze_connect import BreezeConnect 

In [ ]:
STOCK ='VEDL'

In [ ]:
# connecting Breeze API 
api =  BreezeConnect(api_key=api_key)
# Generate Session
api.generate_session(api_secret=api_secret,
                        session_token=api_session)

In [ ]:
today =datetime.now()
fromdate = today-timedelta(days=20)

In [ ]:
# Requesting historical data from broker
data = api.get_historical_data_v2(interval="1minute",
                            from_date= "2022-08-15T07:00:00.000Z",
                            to_date= "2023-08-17T07:00:00.000Z",
                            stock_code='RELIND',
                            exchange_code="NSE",
                            product_type="cash")

In [ ]:
# Shaping data for panda frame
# Adjusting index column and format for backtrader

df =pd.DataFrame(data['Success'])
df =df[['datetime','open','high','low','close','volume']]
df['datetime']= pd.to_datetime(df['datetime']) 
df.set_index('datetime', inplace=True)
df

In [ ]:
# LETS CREATE A STRATEGY AND TEST IT WITH BACK TRADER

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
from datetime import datetime  # For datetime objects
import backtrader as bt
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        try:
            ''' Logging function fot this strategy'''
            dt = dt or self.datas[0].datetime.date(0)
            # dt=pd.to_datetime(dt)
            print('%s, %s' % (dt.isoformat(), txt))
        except:
            print("Got an error")

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)

        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # data = yf.download('RELIANCE.NS', start=datetime(2021, 1, 1), end=datetime(2023, 12, 1))
    data_feed = bt.feeds.PandasData(dataname=df)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data_feed)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    #Analyzer
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    results = cerebro.run()
    strat = results[0]

    portfolio_stats = strat.analyzers.getbyname('PyFolio')
    returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
    # returns.index = returns.index.tz_convert(None)

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Plot the result
    cerebro.plot()

In [ ]:
# CREATE HTML TEAR SHEET - html - quants
import quantstats

quantstats.reports.html(returns, output='stats.html', title='BTC Sentiment')
